# MOSAIC Paper Numerical Verification

**Paper:** "When AI Takes Our Jobs, It Should Also Pay Our Wages"

This notebook verifies paper values against data sources and runs all calculations.

## Organization

Follows `number_hierarchy.md` structure:
- **Layer 1:** Exogenous Inputs (external data, design assumptions)
- **Layer 2:** Calculated Values (formulas from Layer 1)
- **Layer 3:** Simulated Values (numerical solvers, Monte Carlo)

## Cross-references

| Section | number_hierarchy.md | numerical_audit.md |
|---------|---------------------|--------------------|
| 1.1-1.5 | Layer 1 | D01-D12, S01-S04 |
| 2.1-2.6 | Layer 2 | N01-N06, H01-H04, F01-F10, V01-V18 |
| 3.1-3.5 | Layer 3 | B01-B12, FI01-FI08, PE01-PE08 |

In [1]:
# Imports
from typing import Optional
import math

# CALC infrastructure for storing computed values
from functions import init_calc, load_calc, store, get, CALC

# Initialize CALC fresh (clears any previous values)
# This ensures verify_layer1-4 can be run sequentially to replicate all results
init_calc()
print(f"CALC initialized fresh: {sum(len(v) for v in CALC.values())} values (should be 0)")

CALC initialized fresh: 0 values (should be 0)


---
# DATA Dictionary

Verified values from official sources (CBS, BTL, Bank of Israel, Taub Center).

In [2]:
# Import DATA dict from config.py
from config import DATA

# Print summary
print(f'DATA dict loaded: {len(DATA)} items')


DATA dict loaded: 95 items


---
# BTL2025 Deciles Data

Wage decile data from BTL2025.

In [3]:
# Import BTL2025_DECILES from config.py
from config import BTL2025_DECILES

# Print summary
print(f'BTL2025_DECILES loaded: {len(BTL2025_DECILES)} deciles')


BTL2025_DECILES loaded: 10 deciles


---
# Function Library

All calculation functions defined here. Each has:
- Docstring with formula, inputs, outputs
- Type hints
- Single responsibility

In [4]:
# Import verify function (store/get already imported in cell-1)
from functions import verify

print('Layer 1: Stores exogenous inputs only (no calculations)')

Layer 1: Stores exogenous inputs only (no calculations)


---
# LAYER 1: Exogenous Inputs

Numbers from external sources or pure design assumptions. Verify against primary sources.

## 1.1 Israeli Economy Data

**Sources:** BOI 2024, BTL2025, BTL2023, Adva Center 2024

| ID | Description | Value | Source |
|----|-------------|-------|--------|
| B01 | Initial GDP Y0 | 1,694B NIS | BOI 2024 |
| D06 | Labor share sL | 0.56 | Adva 2024 |
| B02 | Labor force | 4.5M | BOI 2024 |
| A03 | Natural unemployment | 3.0% | BOI 2024 |
| D01 | Median wage | 10,586 NIS/mo | BTL2025 |
| D05 | Poverty rate | 20.7% | BTL2023 |
| D06a | Poverty gap | 39.5% | BTL2023Poverty |
| PL01 | Poverty line | 3,324 NIS/mo | BTL2023Poverty |
| G01 | Pre-NIT Gini | 0.363 | BTL2023Poverty |

In [5]:
print("=" * 60)
print("1.1 ISRAELI ECONOMY DATA")
print("=" * 60)

# Store Layer 1 economy values to CALC
economy_items = [
    ("Y0", "Initial GDP (B NIS)"),
    ("labor_share", "Labor share sL"),
    ("labor_force", "Labor force (M)"),
    ("natural_unemployment", "Natural unemployment u0 (%)"),
    ("median_wage", "Median wage (NIS/mo)"),
    ("P80_income", "80th percentile income (NIS/mo)"),
    ("poverty_rate_initial", "Poverty rate (%)"),
    ("poverty_gap_initial", "Poverty gap index (%)"),
    ("poverty_line", "Poverty line (NIS/mo)"),
    ("gini_initial", "Pre-NIT Gini coefficient"),
    ("total_households", "Total households (M)"),
]

print("\nStoring Layer 1 economy values to CALC:")
for name, desc in economy_items:
    if name in DATA:
        value = DATA[name]
        store(name, value, "layer1")
        print(f"  {name}: {value} ({desc})")

1.1 ISRAELI ECONOMY DATA

Storing Layer 1 economy values to CALC:
  Y0: 1694 (Initial GDP (B NIS))
  labor_share: 0.56 (Labor share sL)
  labor_force: 4.5043 (Labor force (M))
  natural_unemployment: 3.0 (Natural unemployment u0 (%))
  median_wage: 10586 (Median wage (NIS/mo))
  P80_income: 21118 (80th percentile income (NIS/mo))
  poverty_rate_initial: 20.7 (Poverty rate (%))
  poverty_gap_initial: 39.5 (Poverty gap index (%))
  poverty_line: 3324 (Poverty line (NIS/mo))
  gini_initial: 0.363 (Pre-NIT Gini coefficient)
  total_households: 2.9794 (Total households (M))


## 1.2 Elasticity Parameters

**Source:** Taub Center 2024 - https://www.taubcenter.org.il/research/labor-supply-elasticity/

**Taub Center Estimates (Israel, 2014-2019):**
- Intensive (hours): Men 0.45, Women 0.56 -> midpoint **0.51**
- Extensive (participation): Men 0.25, Women 0.31 -> midpoint **0.28**

**Weighted Elasticity (Simple Average):**
$$\varepsilon = \frac{\varepsilon_h + \varepsilon_p}{2} = \frac{0.51 + 0.28}{2} = 0.39$$

In [6]:
print("=" * 60)
print("1.2 ELASTICITY PARAMETERS")
print("=" * 60)

# Taub Center 2024 raw values
TAUB = {
    "intensive": {"men": 0.45, "women": 0.56},
    "extensive": {"men": 0.25, "women": 0.31},
}

# Calculate gender midpoints (this is just averaging raw data, not model calculation)
eps_intensive = (TAUB["intensive"]["men"] + TAUB["intensive"]["women"]) / 2
eps_extensive = (TAUB["extensive"]["men"] + TAUB["extensive"]["women"]) / 2

print(f"\nTaub Center 2024 (gender midpoints):")
print(f"  Intensive: ({TAUB['intensive']['men']} + {TAUB['intensive']['women']})/2 = {eps_intensive:.2f}")
print(f"  Extensive: ({TAUB['extensive']['men']} + {TAUB['extensive']['women']})/2 = {eps_extensive:.2f}")

# Store raw elasticity values to Layer 1
store("epsilon_intensive", eps_intensive, "layer1")
store("epsilon_extensive", eps_extensive, "layer1")
store("income_tax_wedge", DATA["income_tax_wedge"], "layer1")

print(f"\nStored to CALC (Layer 1):")
print(f"  epsilon_intensive: {eps_intensive}")
print(f"  epsilon_extensive: {eps_extensive}")
print(f"  income_tax_wedge: {DATA['income_tax_wedge']}")
print(f"\nNote: epsilon_weighted calculated in Layer 2 as (intensive + extensive)/2")

1.2 ELASTICITY PARAMETERS

Taub Center 2024 (gender midpoints):
  Intensive: (0.45 + 0.56)/2 = 0.51
  Extensive: (0.25 + 0.31)/2 = 0.28

Stored to CALC (Layer 1):
  epsilon_intensive: 0.505
  epsilon_extensive: 0.28
  income_tax_wedge: 10

Note: epsilon_weighted calculated in Layer 2 as (intensive + extensive)/2


## 1.3 Scenario Parameters

**Type:** Assumptions (calibrated to literature)

| Scenario | alpha | sL' | u | g_Y |
|----------|-------|-----|---|-----|
| Low Displacement | 1.4 | 0.50 | 6% | 38% |
| Strong | 1.6 | 0.42 | 10% | 53% |
| **AGI** | 2.2 | 0.35 | 40% | 70% |
| **ASI** | 3.0 | 0.25 | 60% | ~120% |

**Note:** g_Y calculated via CES production function with K constant. AGI is the decision scenario; ASI is a stress test.

In [7]:
print("=" * 60)
print("1.3 SCENARIO PARAMETERS (Assumptions)")
print("=" * 60)

# Store scenario assumption parameters to Layer 1 (from DATA)
scenario_params = [
    ("alpha_low_displacement", "alpha_strong", "alpha_agi", "alpha_asi"),
    ("s_L_prime_low_displacement", "s_L_prime_strong", "s_L_prime_agi", "s_L_prime_asi"),
    ("u_low_displacement", "u_strong", "u_agi", "u_asi"),
]

print("\nStoring scenario assumptions to CALC (Layer 1):")
for params in scenario_params:
    for key in params:
        store(key, DATA[key], "layer1")

# Also store CES parameters
store("K_0", DATA["K_0"], "layer1")
store("sigma", DATA["sigma"], "layer1")

scenarios = [
    ("Low Displacement", "alpha_low_displacement", "s_L_prime_low_displacement", "u_low_displacement"),
    ("Strong", "alpha_strong", "s_L_prime_strong", "u_strong"),
    ("AGI", "alpha_agi", "s_L_prime_agi", "u_agi"),
    ("ASI", "alpha_asi", "s_L_prime_asi", "u_asi"),
]

print(f"\n{'Scenario':<10} {'alpha':>6} {'sL_prime':>8} {'u':>6}")
print("-" * 35)
for name, alpha_key, sl_key, u_key in scenarios:
    marker = "**" if name in ("AGI", "ASI") else "  "
    print(f"{marker}{name:<8} {DATA[alpha_key]:>6} {DATA[sl_key]:>8} {DATA[u_key]:>5}%")
print("-" * 35)
print("Note: g_Y is CALCULATED in Layer 2 via CES production function")
print("      AGI = decision scenario; ASI = stress test")

1.3 SCENARIO PARAMETERS (Assumptions)

Storing scenario assumptions to CALC (Layer 1):

Scenario    alpha sL_prime      u
-----------------------------------
  Low Displacement    1.4      0.5     6%
  Strong      1.6     0.42    10%
**AGI         2.2     0.35    40%
**ASI         3.0     0.25    60%
-----------------------------------
Note: g_Y is CALCULATED in Layer 2 via CES production function
      AGI = decision scenario; ASI = stress test


## 1.4 NIT Design Parameters

**Type:** Policy design choices

| ID | Description | Value | Type |
|----|-------------|-------|------|
| N01 | Reference floor M | 6,350 NIS/mo | DESIGN (60% median) |
| N02 | Disregard D | 1,000 NIS/mo | DESIGN |
| N03 | Single-parent top-up psi | 900 NIS/mo | DESIGN |
| N04 | Implemented taper tau | 0.50 | DESIGN |
| B08 | Child equivalence phi | 0.30 | VERIFIED (OECD) |
| TU01 | Take-up rate | 85% | DESIGN |
| TU02 | GDP buffer (100% take-up) | 3% | DESIGN |

In [8]:
print("=" * 60)
print("1.4 NIT DESIGN PARAMETERS")
print("=" * 60)

# Store NIT design parameters to Layer 1 (from DATA)
nit_params = [
    "floor_reference",
    "disregard",
    "single_parent_topup",
    "taper_implemented",
    "child_equiv_weight",
    "nit_take_up_rate",
    "nit_gdp_buffer_pct",
]

print("\nStoring NIT design parameters to CALC (Layer 1):")
for key in nit_params:
    store(key, DATA[key], "layer1")

# Show floor as ~60% of median wage
median_wage = DATA['median_wage']
floor_ref = DATA['floor_reference']
floor_pct = floor_ref / median_wage * 100

print(f"  floor_reference: M = {floor_ref:,} NIS/mo (~{floor_pct:.0f}% of median wage {median_wage:,}) [DESIGN]")
print(f"  disregard: D = {DATA['disregard']:,} NIS/mo [DESIGN]")
print(f"  single_parent_topup: psi = {DATA['single_parent_topup']:,} NIS/mo [DESIGN]")
print(f"  taper_implemented: tau = {DATA['taper_implemented']} [DESIGN]")
print(f"  child_equiv_weight: phi = {DATA['child_equiv_weight']} [VERIFIED: OECD standard]")
print(f"  nit_take_up_rate: {DATA['nit_take_up_rate']}% [DESIGN]")
print(f"  nit_gdp_buffer_pct: {DATA['nit_gdp_buffer_pct']}% [DESIGN]")
print(f"\nNote: taper_optimal is CALCULATED in Layer 2 via Saez formula")

1.4 NIT DESIGN PARAMETERS

Storing NIT design parameters to CALC (Layer 1):
  floor_reference: M = 6,350 NIS/mo (~60% of median wage 10,586) [DESIGN]
  disregard: D = 1,000 NIS/mo [DESIGN]
  single_parent_topup: psi = 900 NIS/mo [DESIGN]
  taper_implemented: tau = 0.5 [DESIGN]
  child_equiv_weight: phi = 0.3 [VERIFIED: OECD standard]
  nit_take_up_rate: 85% [DESIGN]
  nit_gdp_buffer_pct: 3% [DESIGN]

Note: taper_optimal is CALCULATED in Layer 2 via Saez formula


## 1.5 Funding Mechanism Inputs

| ID | Description | Value | Source | Status |
|----|-------------|-------|--------|--------|
| V01 | Baseline VAT | 18% | ITA2024VAT | VERIFIED |
| V02 | BLS deflation rate | 5%/yr | BLSSoftwareCPI | VERIFIED |
| V03 | BLS deflation alpha | 1.4 | Design (baseline) | DESIGN |
| V04 | AI consumption exposure | 40% | Design | DESIGN |
| P01 | Profit trend baseline | 50B NIS | Knesset2021 | CALCULATED |

**Deflation Derivation (Layer 2):**
- Low Displacement (α=1.4): δ = 5% (BLS historical anchor)
- Above baseline: δ = δ_BLS × [1 + 0.40 × (α - α_BLS) / (α_BLS - 1)]

**Calculated in Layer 2:**
- `deflation_*`: From alpha via BLS anchor
- `vat_passthrough`: Event study (§2.5)
- `profit_trend_linear/gdp_linked/baseline`: From corporate tax data

In [9]:
print("=" * 60)
print("1.5 FUNDING MECHANISM INPUTS")
print("=" * 60)

# Store funding mechanism inputs to Layer 1 (from DATA)
funding_params = [
    "vat_baseline",
    # Deflation now derived from alpha in Layer 2 - store BLS anchor params
    "bls_deflation_rate",
    "bls_deflation_alpha",
    "ai_consumption_exposure",
    "private_consumption",
    "consumption_share",
    "ai_attribution_coef",
    "ringfence_rate",
    "corporate_tax_rate",
    "capital_gains_rate",
]

print("\nStoring funding mechanism inputs to CALC (Layer 1):")
for key in funding_params:
    store(key, DATA[key], "layer1")

print(f"  vat_baseline: {DATA['vat_baseline']}%")
print(f"  bls_deflation_rate: {DATA['bls_deflation_rate']}% (BLS historical software deflation)")
print(f"  bls_deflation_alpha: {DATA['bls_deflation_alpha']} (alpha anchor for BLS rate)")
print(f"  ai_consumption_exposure: {DATA['ai_consumption_exposure']}% (fraction of consumption AI-affected)")
print(f"  private_consumption: {DATA['private_consumption']}T NIS")
print(f"  ai_attribution_coef: {DATA['ai_attribution_coef']}%")
print(f"  ringfence_rate: {DATA['ringfence_rate']}%")

print(f"\nNote: deflation_* values are CALCULATED in Layer 2 from alpha")
print(f"      Formula: δ = 0.80 × δ_BLS × (α - 1) / (α_BLS - 1) for α > 1.4")
print(f"      vat_passthrough is CALCULATED in Layer 2 (event study)")
print(f"      profit_trend_linear/gdp_linked/baseline are CALCULATED in Layer 2")

1.5 FUNDING MECHANISM INPUTS

Storing funding mechanism inputs to CALC (Layer 1):
  vat_baseline: 18%
  bls_deflation_rate: 5% (BLS historical software deflation)
  bls_deflation_alpha: 1.4 (alpha anchor for BLS rate)
  ai_consumption_exposure: 40% (fraction of consumption AI-affected)
  private_consumption: 1.0T NIS
  ai_attribution_coef: 100%
  ringfence_rate: 75%

Note: deflation_* values are CALCULATED in Layer 2 from alpha
      Formula: δ = 0.80 × δ_BLS × (α - 1) / (α_BLS - 1) for α > 1.4
      vat_passthrough is CALCULATED in Layer 2 (event study)
      profit_trend_linear/gdp_linked/baseline are CALCULATED in Layer 2


## 1.5a Pre-AI Profit Trend Data (P01)

**Sources:** 
- Corporate Tax: [Knesset Research 2021](https://fs.knesset.gov.il/globaldocs/MMM/858f0142-fcf4-eb11-8114-00155d0aee38/2_858f0142-fcf4-eb11-8114-00155d0aee38_11_18135.pdf)
- GDP: World Bank WDI (USD) × Bank of Israel exchange rates

**Data:** 2015-2019 (pre-COVID, pre-AI period)

| Year | Corporate Tax (B NIS) | GDP (B NIS) | Ratio |
|------|----------------------|-------------|-------|
| 2015 | 32.8 | 1,179 | 2.78% |
| 2016 | 35.8 | 1,236 | 2.89% |
| 2017 | 39.8 | 1,290 | 3.09% |
| 2018 | 40.8 | 1,354 | 3.01% |
| 2019 | 39.9 | 1,428 | 2.79% |

**Calculated in Layer 2:**
- `profit_trend_linear`: Linear extrapolation to 2024
- `profit_trend_gdp_linked`: Avg ratio × current GDP
- `profit_trend_baseline`: Midpoint of above

In [10]:
print("=" * 60)
print("1.5a PRE-AI PROFIT TREND DATA (P01)")
print("=" * 60)

# Corporate tax revenue data (Knesset2021) - raw data stored in Layer 1
years = [2015, 2016, 2017, 2018, 2019]
revenues = [DATA['corp_tax_2015'], DATA['corp_tax_2016'], DATA['corp_tax_2017'],
            DATA['corp_tax_2018'], DATA['corp_tax_2019']]
gdps = [DATA['gdp_2015'], DATA['gdp_2016'], DATA['gdp_2017'],
        DATA['gdp_2018'], DATA['gdp_2019']]

print(f"\nCorporate Tax Revenue (B NIS) - Source: Knesset2021")
print(f"Historical GDP (B NIS) - Source: CBS National Accounts")
print(f"\n{'Year':>6} {'Corp Tax':>10} {'GDP':>10} {'Ratio':>8}")
print("-" * 38)
for y, r, g in zip(years, revenues, gdps):
    ratio = r / g * 100
    print(f"{y:>6} {r:>10.1f} {g:>10.0f} {ratio:>7.2f}%")

# Store raw data values to Layer 1
for y in years:
    store(f'corp_tax_{y}', DATA[f'corp_tax_{y}'], "layer1")
    store(f'gdp_{y}', DATA[f'gdp_{y}'], "layer1")

print(f"\nNote: profit_trend_linear, profit_trend_gdp_linked, and")
print(f"      profit_trend_baseline are CALCULATED in Layer 2")

1.5a PRE-AI PROFIT TREND DATA (P01)

Corporate Tax Revenue (B NIS) - Source: Knesset2021
Historical GDP (B NIS) - Source: CBS National Accounts

  Year   Corp Tax        GDP    Ratio
--------------------------------------
  2015       32.8       1179    2.78%
  2016       35.8       1236    2.89%
  2017       39.8       1290    3.09%
  2018       40.8       1354    3.01%
  2019       39.9       1428    2.79%

Note: profit_trend_linear, profit_trend_gdp_linked, and
      profit_trend_baseline are CALCULATED in Layer 2


## 1.5b Ring-Fencing Parameters (P03-P04)

**Type:** Design parameters for ring-fenced profit calculation

**Parameters:**
- κ = 1.0 (AI attribution coefficient, `ai_attribution_coef`)
- ρ = 0.75 (ring-fence rate, unified for all scenarios)

**Note:** Ring-fenced profit calculation in Layer 2 Section 2.5b

**Formula (applied in Layer 2):**
- Projected = Baseline × (1 + g_Y)
- Over-trend = Projected - Baseline
- Ring-fenced = κ × ρ × Over-trend

In [11]:
print("=" * 60)
print("1.5b RING-FENCING PARAMETERS")
print("=" * 60)

# Document the parameters - calculation happens in Layer 2
print(f"\nParameters for ring-fenced profit calculation (stored in Layer 1):")
print(f"  ai_attribution_coef: {get('ai_attribution_coef')}% (κ=1)")
print(f"  ringfence_rate: {get('ringfence_rate')}% (unified for all scenarios)")
print(f"\nNote: profit_trend_* values calculated in Layer 2 from raw corp_tax data")
print(f"      Ring-fenced profit calculations are in Layer 2 Section 2.5b")

1.5b RING-FENCING PARAMETERS

Parameters for ring-fenced profit calculation (stored in Layer 1):
  ai_attribution_coef: 100% (κ=1)
  ringfence_rate: 75% (unified for all scenarios)

Note: profit_trend_* values calculated in Layer 2 from raw corp_tax data
      Ring-fenced profit calculations are in Layer 2 Section 2.5b


## 1.5c Capital Gains Tax Data (CG01)

**Source:** [Knesset Research Center 2024](https://fs.knesset.gov.il/globaldocs/MMM/ed365cc9-82af-ee11-8162-005056aa4246/2_ed365cc9-82af-ee11-8162-005056aa4246_11_20755.pdf)

**Data:** Capital gains tax revenue 2012-2019 (dividend + securities + net interest)

| Year | Total CG Tax (B NIS) | Note |
|------|---------------------|------|
| 2012 | 10.2 | Post-2012 tax reform |
| 2013 | 7.5 | |
| 2014 | 7.3 | |
| 2015 | 8.4 | |
| 2016 | 7.1 | |
| 2017 | 19.6 | **OUTLIER** (dividend spike) |
| 2018 | 7.2 | |
| 2019 | 7.7 | |

**Calculated in Layer 2:**
- `cg_trend_linear`: Linear extrapolation to 2024 (excluding 2017)
- `cg_trend_gdp_linked`: Avg ratio × current GDP
- `cg_trend_baseline`: Midpoint of above

In [12]:
print("=" * 60)
print("1.5c CAPITAL GAINS TAX DATA (CG01)")
print("=" * 60)

# Capital gains tax revenue data (Knesset Research Center 2024)
# Source: https://fs.knesset.gov.il/globaldocs/MMM/ed365cc9-82af-ee11-8162-005056aa4246/2_ed365cc9-82af-ee11-8162-005056aa4246_11_20755.pdf
cg_years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
cg_revenues = [DATA['cg_tax_2012'], DATA['cg_tax_2013'], DATA['cg_tax_2014'],
               DATA['cg_tax_2015'], DATA['cg_tax_2016'], DATA['cg_tax_2017'],
               DATA['cg_tax_2018'], DATA['cg_tax_2019']]

print(f"\nCapital Gains Tax Revenue (B NIS) - Source: Knesset2024CG")
print(f"Components: Dividend + Securities + Net Interest")
print(f"\n{'Year':>6} {'CG Tax':>10} {'Note':>20}")
print("-" * 40)
for y, r in zip(cg_years, cg_revenues):
    note = "OUTLIER (dividend spike)" if y == 2017 else ""
    print(f"{y:>6} {r:>10.1f} {note:>20}")

# Store raw data values to Layer 1
for y in cg_years:
    store(f'cg_tax_{y}', DATA[f'cg_tax_{y}'], "layer1")

# Also store historical GDP 2012-2019 for GDP-linked calculation (if not already stored)
# Note: GDP 2015-2019 already stored in 1.5a
gdp_2012_2014 = {2012: 1029, 2013: 1078, 2014: 1120}  # CBS National Accounts
for y, g in gdp_2012_2014.items():
    store(f'gdp_{y}', g, "layer1")

print(f"\nNote: cg_trend_linear, cg_trend_gdp_linked, and")
print(f"      cg_trend_baseline are CALCULATED in Layer 2 (Section 2.5e)")
print(f"\n      2017 EXCLUDED from trend calculation (outlier year)")

1.5c CAPITAL GAINS TAX DATA (CG01)

Capital Gains Tax Revenue (B NIS) - Source: Knesset2024CG
Components: Dividend + Securities + Net Interest

  Year     CG Tax                 Note
----------------------------------------
  2012       10.2                     
  2013        7.5                     
  2014        7.3                     
  2015        8.4                     
  2016        7.1                     
  2017       19.6 OUTLIER (dividend spike)
  2018        7.2                     
  2019        7.7                     

Note: cg_trend_linear, cg_trend_gdp_linked, and
      cg_trend_baseline are CALCULATED in Layer 2 (Section 2.5e)

      2017 EXCLUDED from trend calculation (outlier year)


## 1.6 Government Ring-fencing Inputs (G01-G03)

**Type:** Exogenous inputs (calculation in Layer 2)

**Inputs:**
- Personnel Cost: 203B NIS (MOF2024Financial)
- Substitutable Share: 40.3% (MOSAIC2024AI + internal projection onto public sector)
- Realization Rate: 10% (Low Displacement), 30% (Strong), 50% (AGI)
- Agency Retention: 25% (design parameter)

**Note:** Government ring-fencing calculation moved to Layer 2 Section 2.5c

In [13]:
print("=" * 60)
print("1.6 GOVERNMENT RING-FENCING INPUTS (G01-G03)")
print("=" * 60)

# Store government ring-fencing inputs to Layer 1 (from DATA)
# Note: GAD now uses unemployment rate (u) directly, not separate realization rates
gad_inputs = [
    "public_sector_wages",
    "public_sector_substitutable",
]

print("\nStoring government ring-fencing inputs to CALC (Layer 1):")
for key in gad_inputs:
    store(key, DATA[key], "layer1")

print(f"  public_sector_wages: {DATA['public_sector_wages']}B NIS (MOF2024Financial)")
print(f"  public_sector_substitutable: {DATA['public_sector_substitutable']}% (MOSAIC2024AI + internal)")
print(f"\nNote: Government ring-fencing formula uses unemployment rate (u) directly:")
print(f"      R_gov = W × u × ρ")
print(f"      See Layer 2 Section 2.5c for calculation")

1.6 GOVERNMENT RING-FENCING INPUTS (G01-G03)

Storing government ring-fencing inputs to CALC (Layer 1):
  public_sector_wages: 203B NIS (MOF2024Financial)
  public_sector_substitutable: 40.3% (MOSAIC2024AI + internal)

Note: Government ring-fencing formula uses unemployment rate (u) directly:
      R_gov = W × u × ρ
      See Layer 2 Section 2.5c for calculation


## 1.7 Program Consolidation Inputs (C01)

**Type:** Exogenous inputs (calculation in Layer 2)

**Inputs (BTL2023 Annual Report):**
- Income Support (הבטחת הכנסה): 1,643M NIS = 1.643B NIS
- Unemployment (דמי אבטלה): 5,563M NIS = 5.563B NIS
- Admin Savings Rate: 3%

**Note:** Consolidation revenue calculation moved to Layer 2 Section 2.5d

In [14]:
print("=" * 60)
print("1.7 PROGRAM CONSOLIDATION INPUTS (C01)")
print("=" * 60)

# Store consolidation inputs to Layer 1 (from DATA)
consolidation_inputs = [
    "btl_income_support",
    "btl_unemployment",
    "consolidation_admin_rate",
]

print("\nStoring consolidation inputs to CALC (Layer 1):")
for key in consolidation_inputs:
    store(key, DATA[key], "layer1")

print(f"\nInputs (BTL2023 Annual Report):")
print(f"  Income Support (הבטחת הכנסה): {DATA['btl_income_support']:.3f}B NIS")
print(f"  Unemployment (דמי אבטלה): {DATA['btl_unemployment']:.3f}B NIS")
print(f"  Admin Savings Rate: {DATA['consolidation_admin_rate']}%")
print(f"\nNote: Consolidation revenue is CALCULATED in Layer 2")

1.7 PROGRAM CONSOLIDATION INPUTS (C01)

Storing consolidation inputs to CALC (Layer 1):

Inputs (BTL2023 Annual Report):
  Income Support (הבטחת הכנסה): 1.643B NIS
  Unemployment (דמי אבטלה): 5.563B NIS
  Admin Savings Rate: 3%

Note: Consolidation revenue is CALCULATED in Layer 2
